### Recommendation System 

Recommendation systems are indeed a type of unsupervised learning that filter and predict items or content that a user might like based on their preferences or similarities to other users. They are widely used in various applications such as e-commerce, streaming platforms, social media, and more.
> it's work on finding angle or distance between each data point to other using cosine similarity or Euclidean formula and give predict nearest value.
> for e.g. a dataset contains three values and we have to find the similarity between them so i use cosine similarity between each sample to others sample.
> if value or angle between them is going to -1 then both sample does not similar if value goes to +1 then both value is similar if value is 0 then both value is neutarl.
> cosine similarity find the angle between them.

## Types of Recommendations System.
> 1.  Content-Based Filtering:- This approach recommends items based on the characteristics and features of the items themselves. It works by analyzing the attributes of items that a user has liked or interacted with and then recommending similar items based on those attributes. Content-based filtering focuses on item descriptions, keywords, or other metadata.
> 2.  Collaborative Filtering:- This type of recommendation system recommends items based on the preferences of similar users. It works by identifying users with similar tastes or preferences and recommending items that those similar users have liked or consumed. Collaborative filtering can be further divided into two approaches:

       > 1. User-Based Collaborative Filtering: Recommends items based on the preferences and behavior of similar users.
       > 2. Item-Based Collaborative Filtering: Recommends items that are similar to those previously liked or consumed by the user.
> 3. Hybrid Recommendation Systems:- These systems combine collaborative filtering and content-based filtering (and sometimes other techniques) to provide more accurate and effective recommendations. Hybrid systems leverage the strengths of both approaches to overcome their individual limitations and improve recommendation quality.

### how to find similarty b/w 2 docs

> 1. obtain vector representation of each doc using any vectorizer technique.
> 2. use euclidean or cosine metric to find similiarty scores(cosine is prefered)

## If similarity Score is 0 or goes to -1 tham both words not similar and if similarity score goes to +1 than both word is similar.

### Content Based Filtering.

In [479]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer,ENGLISH_STOP_WORDS
import warnings
warnings.filterwarnings('ignore')

In [481]:
s1="python is a Programming language"
s2="india lost world cup 2023"
s3="java was trending programming Language"

In [483]:
tf=TfidfVectorizer(stop_words=sw)
X=tf.fit_transform([s1,s2,s3]).toarray()
sim=cosine_similarity(X)
dis=cosine_distances(X)
print("Similarity Socre:- ",sim)
print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
print("Similarity Distance:- ",dis)

Similarity Socre:-  [[1.         0.         0.44333251]
 [0.         1.         0.        ]
 [0.44333251 0.         1.        ]]
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
Similarity Distance:-  [[0.         1.         0.55666749]
 [1.         0.         1.        ]
 [0.55666749 1.         0.        ]]


In [485]:
df=pd.read_csv("G:/dataset/recommend/movies_content.csv")
# df=df[['name','description']] # Alternative of iloc
df=df.iloc[:,[-1,1]]
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True,inplace=True)
sw=list(ENGLISH_STOP_WORDS)
tf=TfidfVectorizer(lowercase=True,stop_words=sw)
X=tf.fit_transform(df['description']).toarray()
# len(tf.get_feature_names_out())  # length = 7702
model=NearestNeighbors(metric='cosine')
model.fit(X)

NearestNeighbors(metric='cosine')

In [487]:
name='Neerja'
print(model.kneighbors([X[0]],n_neighbors=6))
print(df.iloc[309][1])
df.iloc[309]

(array([[0.        , 0.85030796, 0.87983038, 0.92082846, 0.93608081,
        0.93927533]]), array([[   0, 1327,  107, 1563,  560,  309]], dtype=int64))
A doctor of Indian origin returning to Mumbai from London is forced to participate in a terrorist plot.


name                                                       Aamir
description    A doctor of Indian origin returning to Mumbai ...
Name: 309, dtype: object

In [489]:
try:
    movie_name=input("Enter Movie Name:- ")
    idx=df[df.name==movie_name].index[0]
    score,index=model.kneighbors([X[idx]],n_neighbors=6)
    movie=[]
    for i in index[0][1:]:
        movie.append(df.iloc[i][0])
    df_1=pd.DataFrame(data=movie,columns=['name'])
    print("---------Top 5 Recommended Movies----------")
    print("--------------------------------------------")
    print(df_1)
except:
    print("Movie Not Found!")

Enter Movie Name:-  PK


---------Top 5 Recommended Movies----------
--------------------------------------------
                            name
0               101 Chodhyangal?
1                        Kahaani
2                Ship of Theseus
3  Haggada Kone: End of the Rope
4             Aa Te Kevi Dunniya


### Collaborative Based Filtering.

In [493]:
df=pd.read_csv("G:/dataset/recommend/movies_collaborative.csv")
user_movie=df.pivot_table(index='userId',columns='movieId',values='rating')
user_movie.fillna(0,inplace=True)
model=NearestNeighbors(metric='cosine')
X=user_movie.values
model.fit(X)

NearestNeighbors(metric='cosine')

In [495]:
model.kneighbors([X[0]],n_neighbors=5)
sim_movie=df[df['userId']==266].movieId.values
target_movie=df[df['userId']==1].movieId.values
recommend_movie=set(sim_movie)-set(target_movie)
df_recomnd_movie=df[(df.userId==266) & df.movieId.isin(recommend_movie)]
df_recomnd_movie=df_recomnd_movie.sort_values('rating',ascending=False)
df_recomnd_movie.head()

,movieId,title,userId,rating
903,16,Casino (1995),266,5.0
18769,778,Trainspotting (1996),266,5.0
22888,1079,"Fish Called Wanda, A (1988)",266,5.0
25215,1199,Brazil (1985),266,5.0
25309,1200,Aliens (1986),266,5.0


In [499]:
df=pd.read_csv("G:/dataset/recommend/movies_collaborative.csv")
user_movie=df.pivot_table(index='userId',columns='movieId',values='rating')
user_movie.fillna(0,inplace=True)
model=NearestNeighbors(metric='cosine')
X=user_movie.values
model.fit(X)
try:
    user_id=int(input("Please Enter Your User Id:-"))
    movies=[]
    rating=[]
    score,idx=model.kneighbors([X[user_id]],n_neighbors=2)
    target_userid=idx[0][1]+1
    user_id=idx[0][0]+1
    sim_movie=df[df['userId']==target_userid].movieId.values
    target_movie=df[df['userId']==user_id].movieId.values
    recommend_movie_set=set(sim_movie)-set(target_movie)
    df_recomnd_movie=df[(df.userId==target_userid) & df.movieId.isin(recommend_movie)]
    df_recomnd_movie=df_recomnd_movie.sort_values('rating',ascending=False).head()
    for i,j in zip(df_recomnd_movie.title,df_recomnd_movie.rating):
        movies.append(i)
        rating.append(j)
    d={"Title":movies,"Rating":rating}
    final_df=pd.DataFrame(data=d)
    print("Similar User:",target_userid)
    print("---------Top 5 Recommended Movies----------")
    print(final_df)
except:
    print("User Not Found!")

Please Enter Your User Id:- 3


Similar User: 391
---------Top 5 Recommended Movies----------
                                          Title  Rating
0  Austin Powers: The Spy Who Shagged Me (1999)     5.0
1                           Blade Runner (1982)     5.0
2                           True Romance (1993)     5.0
3                          Trainspotting (1996)     5.0
4                  2001: A Space Odyssey (1968)     5.0
